In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
!pip install pypots>=0.4

In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 22.4 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
import datetime
import numpy as np


import sklearn
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error

import warnings
import xgboost as xgb
import catboost as catt
from catboost import CatBoostRegressor
import math


# Suppress PerformanceWarning
warnings.filterwarnings('ignore')

import uuid
from sklearn.model_selection import KFold



def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True)


In [ ]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [ ]:
os.chdir('/content/drive/MyDrive/EverythingDataScience/AirQuality')
print(f'current directory:  {os.getcwd()}')

current directory:  /content/drive/MyDrive/EverythingDataScience/AirQuality


In [ ]:
train_df = pd.read_csv(os.getcwd() + '/Train.csv')
test_df = pd.read_csv(os.getcwd() + '/Test.csv')
sub = pd.read_csv(os.getcwd() + '/SampleSubmission.csv')

In [ ]:
train_df.head()

,id,site_id,site_latitude,site_longitude,city,country,date,hour,sulphurdioxide_so2_column_number_density,sulphurdioxide_so2_column_number_density_amf,...,cloud_cloud_top_height,cloud_cloud_base_pressure,cloud_cloud_base_height,cloud_cloud_optical_depth,cloud_surface_albedo,cloud_sensor_azimuth_angle,cloud_sensor_zenith_angle,cloud_solar_azimuth_angle,cloud_solar_zenith_angle,pm2_5
0,id_vjcx08sz91,6531a46a89b3300013914a36,6.53257,3.39936,Lagos,Nigeria,2023-10-25,13,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.015000
1,id_bkg215syli,6531a46a89b3300013914a36,6.53257,3.39936,Lagos,Nigeria,2023-11-02,12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.267200
2,id_oui2pot3qd,6531a46a89b3300013914a36,6.53257,3.39936,Lagos,Nigeria,2023-11-03,13,NaN,NaN,...,6791.682888,51171.802486,5791.682829,11.816715,0.192757,-96.411890,61.045123,-121.307414,41.898269,39.450741
3,id_9aandqzy4n,6531a46a89b3300013914a36,6.53257,3.39936,Lagos,Nigeria,2023-11-08,14,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.537600
4,id_ali5x2m4iw,6531a46a89b3300013914a36,6.53257,3.39936,Lagos,Nigeria,2023-11-09,13,0.000267,0.774656,...,1451.050659,96215.906250,451.050598,10.521009,0.153114,-97.811241,49.513439,-126.064453,40.167355,19.431731


In [ ]:
train_df.isnull().sum().tail(40)

formaldehyde_hcho_slant_column_number_density    3915
formaldehyde_cloud_fraction                      3915
formaldehyde_solar_zenith_angle                  3915
formaldehyde_solar_azimuth_angle                 3915
formaldehyde_sensor_zenith_angle                 3915
formaldehyde_sensor_azimuth_angle                3915
uvaerosolindex_absorbing_aerosol_index              5
uvaerosolindex_sensor_altitude                      5
uvaerosolindex_sensor_azimuth_angle                 5
uvaerosolindex_sensor_zenith_angle                  5
uvaerosolindex_solar_azimuth_angle                  5
uvaerosolindex_solar_zenith_angle                   5
ozone_o3_column_number_density                    112
ozone_o3_column_number_density_amf                112
ozone_o3_slant_column_number_density              112
ozone_o3_effective_temperature                    112
ozone_cloud_fraction                              112
ozone_sensor_azimuth_angle                        112
ozone_sensor_zenith_angle   

In [ ]:
train_df.isnull().sum().head(40)

id                                                             0
site_id                                                        0
site_latitude                                                  0
site_longitude                                                 0
city                                                           0
country                                                        0
date                                                           0
hour                                                           0
sulphurdioxide_so2_column_number_density                    4912
sulphurdioxide_so2_column_number_density_amf                4912
sulphurdioxide_so2_slant_column_number_density              4912
sulphurdioxide_cloud_fraction                               4912
sulphurdioxide_sensor_azimuth_angle                         4912
sulphurdioxide_sensor_zenith_angle                          4912
sulphurdioxide_solar_azimuth_angle                          4912
sulphurdioxide_solar_zeni

In [ ]:
nan_cols = ['uvaerosolindex_absorbing_aerosol_index']
for col in nan_cols:
    while train_df[col].isnull().sum()>0:
        train_df[col].fillna(train_df[["site_id", col]].groupby(["site_id"]).shift(periods=0).fillna(method='ffill', limit=1).fillna(method='bfill', limit=1)[col], inplace=True)



In [ ]:
train_df['date'].value_counts().head(20)

date
2024-01-26    60
2024-01-24    59
2024-01-27    59
2024-01-25    57
2024-01-16    54
2023-12-31    54
2024-01-14    52
2023-12-12    51
2023-12-11    49
2023-12-10    47
2023-12-14    47
2023-12-09    47
2024-01-15    47
2023-12-01    46
2023-12-15    45
2023-12-04    44
2023-11-18    43
2023-12-06    43
2023-12-05    42
2023-11-16    42
Name: count, dtype: int64

In [ ]:
train_df.head(40)

,id,site_id,site_latitude,site_longitude,city,country,date,hour,sulphurdioxide_so2_column_number_density,sulphurdioxide_so2_column_number_density_amf,...,cloud_cloud_top_height,cloud_cloud_base_pressure,cloud_cloud_base_height,cloud_cloud_optical_depth,cloud_surface_albedo,cloud_sensor_azimuth_angle,cloud_sensor_zenith_angle,cloud_solar_azimuth_angle,cloud_solar_zenith_angle,pm2_5
0,id_vjcx08sz91,6531a46a89b3300013914a36,6.532570,3.399360,Lagos,Nigeria,2023-10-25,13,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.015000
1,id_bkg215syli,6531a46a89b3300013914a36,6.532570,3.399360,Lagos,Nigeria,2023-11-02,12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.267200
2,id_oui2pot3qd,6531a46a89b3300013914a36,6.532570,3.399360,Lagos,Nigeria,2023-11-03,13,NaN,NaN,...,6791.682888,51171.802486,5791.682829,11.816715,0.192757,-96.411890,61.045123,-121.307414,41.898269,39.450741
3,id_9aandqzy4n,6531a46a89b3300013914a36,6.532570,3.399360,Lagos,Nigeria,2023-11-08,14,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.537600
4,id_ali5x2m4iw,6531a46a89b3300013914a36,6.532570,3.399360,Lagos,Nigeria,2023-11-09,13,0.000267,0.774656,...,1451.050659,96215.906250,451.050598,10.521009,0.153114,-97.811241,49.513439,-126.064453,40.167355,19.431731
5,id_iwv3za16vc,6531a46a89b3300013914a36,6.532570,3.399360,Lagos,Nigeria,2023-11-10,13,-0.000130,0.683252,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.032250
6,id_jsxdghr248,6531a46a89b3300013914a36,6.532570,3.399360,Lagos,Nigeria,2023-11-11,13,0.000018,1.093447,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.417667
7,id_275xmh4mpv,6531a46a89b3300013914a36,6.532570,3.399360,Lagos,Nigeria,2023-11-15,13,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,272.091429
8,id_ym8cixcegi,6531a46a89b3300013914a36,6.532570,3.399360,Lagos,Nigeria,2023-11-16,13,0.000163,0.334319,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,123.559000
9,id_02mf0t6b64,6531a46a89b3300013914a36,6.532570,3.399360,Lagos,Nigeria,2023-11-17,12,0.000114,0.595891,...,4058.409668,71396.875000,3058.409668,4.432884,0.214505,73.518120,38.779526,-142.403320,32.381641,154.909800


In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from pygrinder import mcar
from pypots.data import load_specific_dataset
from pypots.imputation import SAITS
from pypots.utils.metrics import calc_mae

# Data preprocessing. Tedious, but PyPOTS can help.
data = load_specific_dataset('physionet_2012')  # PyPOTS will automatically download and extract it.
X = data['X']
num_samples = len(X['RecordID'].unique())
X = X.drop(['RecordID', 'Time'], axis = 1)
X = StandardScaler().fit_transform(X.to_numpy())
X = X.reshape(num_samples, 48, -1)
X_ori = X  # keep X_ori for validation
X = mcar(X, 0.1)  # randomly hold out 10% observed values as ground truth
dataset = {"X": X}  # X for model input
print(X.shape)  # (11988, 48, 37), 11988 samples and each sample has 48 time steps, 37 features

# # Model training. This is PyPOTS showtime.
# saits = SAITS(n_steps=48, n_features=37, n_layers=2, d_model=256, d_ffn=128, n_heads=4, d_k=64, d_v=64, dropout=0.1, epochs=10)
# # Here I use the whole dataset as the training set because ground truth is not visible to the model, you can also split it into train/val/test sets
# saits.fit(dataset)
# imputation = saits.impute(dataset)  # impute the originally-missing values and artificially-missing values
# indicating_mask = np.isnan(X) ^ np.isnan(X_ori)  # indicating mask for imputation error calculation
# mae = calc_mae(imputation, np.nan_to_num(X_ori), indicating_mask)  # calculate mean absolute error on the ground truth (artificially-missing values)

2024-06-04 05:56:10 [INFO]: Loading the dataset physionet_2012 with TSDB (https://github.com/WenjieDu/Time_Series_Data_Beans)...
2024-06-04 05:56:10 [INFO]: Starting preprocessing physionet_2012...
2024-06-04 05:56:10 [INFO]: You're using dataset physionet_2012, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/physionet_2012
2024-06-04 05:56:10 [INFO]: Dataset physionet_2012 has already been downloaded. Processing directly...
2024-06-04 05:56:10 [INFO]: Dataset physionet_2012 has already been cached. Loading from cache directly...
2024-06-04 05:56:10 [INFO]: Loaded successfully!


KeyError: 'X'

In [ ]:
import numpy as np
from pygrinder import mcar, mar_logistic, mnar_x, mnar_t

# given a time-series dataset with 128 samples, each sample with 10 time steps and 36 data features
ts_dataset = np.random.randn(128, 10, 36)

# grind the dataset with MCAR pattern, 10% missing probability, and using 0 to fill missing values
X_with_mcar_data = mcar(ts_dataset, p=0.1)

### Data Cleaning and Preprocessing

In [ ]:
# concatenate the test and train dataset together
test_df['pm2_5'] = 0
df = pd.concat([train_df, test_df]).reset_index(drop = True)
print(f"shape of dataframe is:{df.shape}")

shape of dataframe is:(10854, 80)


In [ ]:
# drop columns with nan values greater than 70%
dropCol = (df.isnull().sum()/len(df))[(df.isnull().sum()/len(df)).gt(0.70)].index.to_list()
len(dropCol)

7

In [ ]:
# drop these columns with nan values greater than 70% from train and test
df = df.drop(dropCol, axis = 1).reset_index(drop = True)

### Checking if SITE ID and CITY are the same

- we know the cities in train are not the cities in test
- we know that the site_ids in test is different from the ones in train

In [ ]:
print(train_df.site_id.unique().shape, train_df.city.unique().shape, train_df.country.unique().shape, train_df.site_latitude.unique().shape)
print(test_df.site_id.unique().shape, test_df.city.unique().shape, test_df.country.unique().shape)

(69,) (4,) (4,) (68,)
(39,) (4,) (4,)


In [ ]:
train_df.city.unique(), train_df.country.unique()

(array(['Lagos', 'Nairobi', 'Bujumbura', 'Kampala'], dtype=object),
 array(['Nigeria', 'Kenya', 'Burundi', 'Uganda'], dtype=object))

### Data Arrangement
- How is the data in the train and test dataframe arranged in terms of date, do we have a kind of like timeseries prediction into the future on the test set i.e the date in the test is not in the train or it is made up of mixed dates.

In [ ]:
train_df.date.max(),train_df.date.min(), test_df.date.max(),test_df.date.min()

('2024-02-26', '2023-01-01', '2024-02-24', '2023-05-01')

### Filling the Nan values
- The method we want to use is to sort by site_id first and then sort by date and then use forward-fill and backward-fill for the nan values.

In [ ]:
# sort by date
df.sort_values(by=['date']).reset_index(inplace=True, drop=True)

# get the nan columns
idle_cols = ['id', 'site_id', 'city', 'country', 'date', 'site_latitude', 'site_longitude']
nan_cols = df.columns[df.isnull().any()].tolist()
print(len(nan_cols))
nan_cols = [col for col in nan_cols if col not in idle_cols]
print(len(nan_cols))

for col in nan_cols:
    while df[col].isnull().sum()>0:
        df[col].fillna(df[["site_id", col]].groupby(["site_id"]).shift(periods=0).fillna(method='ffill', limit=1).fillna(method='bfill', limit=1)[col], inplace=True)

df.isnull().sum()[df.isnull().sum()>0]

63
63


Series([], dtype: int64)

### Date features

In [ ]:
df["date"] = pd.to_datetime(df.date)

### Categorical from Numerical Features

In [ ]:
not_used_cols = ['id', 'pm2_5']
cat_features = [col for col in df.columns if df[col].dtype == 'object' and col!= 'id']
train_cols = [col for col in df.columns if col not in not_used_cols]

### Modelling

In [ ]:
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class GroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_size : int, default=None
        Maximum size for a single training set.
    Examples
    --------
    >>> import numpy as np
    >>> from sklearn.model_selection import GroupTimeSeriesSplit
    >>> groups = np.array(['a', 'a', 'a', 'a', 'a', 'a',\
                           'b', 'b', 'b', 'b', 'b',\
                           'c', 'c', 'c', 'c',\
                           'd', 'd', 'd'])
    >>> gtss = GroupTimeSeriesSplit(n_splits=3)
    >>> for train_idx, test_idx in gtss.split(groups, groups=groups):
    ...     print("TRAIN:", train_idx, "TEST:", test_idx)
    ...     print("TRAIN GROUP:", groups[train_idx],\
                  "TEST GROUP:", groups[test_idx])
    TRAIN: [0, 1, 2, 3, 4, 5] TEST: [6, 7, 8, 9, 10]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a']\
    TEST GROUP: ['b' 'b' 'b' 'b' 'b']
    TRAIN: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] TEST: [11, 12, 13, 14]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a' 'b' 'b' 'b' 'b' 'b']\
    TEST GROUP: ['c' 'c' 'c' 'c']
    TRAIN: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]\
    TEST: [15, 16, 17]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a' 'b' 'b' 'b' 'b' 'b' 'c' 'c' 'c' 'c']\
    TEST GROUP: ['d' 'd' 'd']
    """
    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_size=None
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_size = max_train_size

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))
        group_test_size = n_groups // n_folds
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []
            for train_group_idx in unique_groups[:group_test_start]:
                train_array_tmp = group_dict[train_group_idx]
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)
            train_end = train_array.size
            if self.max_train_size and self.max_train_size < train_end:
                train_array = train_array[train_end -
                                          self.max_train_size:train_end]
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)
            yield [int(i) for i in train_array], [int(i) for i in test_array]

### Separate Train and Test set

In [ ]:
train, test = df[df['pm2_5'] != 0], df[df['pm2_5'] == 0]
train.shape, test.shape, train_df.shape, test_df.shape

((8071, 73), (2783, 73), (8071, 80), (2783, 80))

In [ ]:
X = train[train_cols]
y = train['pm2_5']


In [ ]:
class PARAM:

  SEED = 42

  n_splits = 10

  params_xgb = {
            'gpu_id': 0,
            #'n_gpus': 2,
            'objective': 'reg:squarederror',
            'eval_metric': 'rmse',
            'booster': 'gbtree',
            'n_estimators': 6000,
            'tree_method': 'gpu_hist',
            'grow_policy': 'lossguide',
            'max_depth': 8,
           'learning_rate': 0.01,
            'max_bin': 90, #
            'max_leaves': 90, #
            'reg_alpha': 8,
            'reg_lambda': 20,
            'subsample': 0.8
            }



folds = KFold(n_splits= PARAM.n_splits, random_state= PARAM.SEED, shuffle = True)

In [ ]:
train_preds = np.zeros(train.shape[0])
test_preds = np.zeros(test.shape[0])


for fold_, (train_idx, val_idx) in enumerate(folds.split(X.values, y)):
    print(f'Fold {fold_+1} / {PARAM.n_splits}' )
    X_train, X_val, X_test = X.iloc[train_idx], X.iloc[val_idx], test
    y_train, y_val = np.log(y.iloc[train_idx]), np.log(y.iloc[val_idx])

    clf = CatBoostRegressor(n_estimators=5000,eval_metric='RMSE',
                                learning_rate=0.175, random_seed= 42,
                                use_best_model=True)

    # clf = CatBoostRegressor(eval_metric='RMSE',
    #                         random_seed= 42,
    #                             use_best_model=True)

    clf.fit(X_train,y_train,eval_set=[(X_val, y_val)],
            cat_features = cat_features,
            early_stopping_rounds=200,verbose=200)


    predTrain = np.exp(clf.predict(X_val))
    train_preds[val_idx] = predTrain
    print(f"fold {fold_ + 1} RMSE : {rmse(y.iloc[val_idx], predTrain)}")

    predTest = np.exp(clf.predict(X_test[train_cols]))
    predTest[predTest < 0] = 0
    test_preds += predTest


test_preds = test_preds / 10
print(f"CV RMSE : {rmse(y, train_preds)}")

Fold 1 / 10
0:	learn: 0.6065823	test: 0.6022786	best: 0.6022786 (0)	total: 15.8ms	remaining: 1m 18s
200:	learn: 0.2439998	test: 0.3289301	best: 0.3289301 (200)	total: 2.58s	remaining: 1m 1s
400:	learn: 0.1872899	test: 0.3114118	best: 0.3114118 (400)	total: 4.65s	remaining: 53.3s
600:	learn: 0.1537056	test: 0.3055703	best: 0.3054304 (596)	total: 6.91s	remaining: 50.6s
800:	learn: 0.1306387	test: 0.3043692	best: 0.3042400 (728)	total: 9.07s	remaining: 47.6s
1000:	learn: 0.1139017	test: 0.3033863	best: 0.3033190 (965)	total: 11.2s	remaining: 44.8s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.303318993
bestIteration = 965

Shrink model to first 966 iterations.
fold 1 RMSE : 11.855966914461984
Fold 2 / 10
0:	learn: 0.5988430	test: 0.6074823	best: 0.6074823 (0)	total: 12.8ms	remaining: 1m 4s
200:	learn: 0.2465814	test: 0.3023991	best: 0.3023991 (200)	total: 2.04s	remaining: 48.8s
400:	learn: 0.1896765	test: 0.2860621	best: 0.2860621 (400)	total: 4.21s	remaining: 48.3s

In [ ]:
sub_xgb = pd.DataFrame()
sub_xgb["ID"] = X_test['id']
sub_xgb["pm2_5"] = test_preds
sub_xgb.to_csv('bfill_ffill_baseline_kfold_cv.csv', index=False)